<a href="https://colab.research.google.com/github/mariajosemv/Predicciones-con-Redes-Neuronales/blob/main/intro_deep_learning/ingenieria_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contexto analítico

Este caso se basa en un reto propuesto en la plataforma [Kaggle](https://www.kaggle.com/austinreese/craigslist-carstrucks-data).

El mercado de autos usados es reconocido por ser un sector economico muy competido con un centenar de compañias que luchan por hacerse con una porción de la torta. El precio de los autos se devalua año año debido a multiples factores y determinar el precio correcto es clave para las compañias para lograr competir en el mercado. En este caso se requiere implementar una red neural que permita determinar el valor más justo para los vehiculos dependiento de sus atributos.

Se cuenta con un dataset (`craiglist_cars.csv`) que serán cargados directamente a Colab luego de su compresión. 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd '/content/drive/MyDrive/Colab Notebooks/Platzi/curso-redes-neuronales/github/intro_deep_learning'
%ls

/content/drive/MyDrive/Colab Notebooks/Platzi/curso-redes-neuronales/github/intro_deep_learning
datasets/  preparacion-de-los-datos.ipynb


In [5]:
import pandas as pd
import numpy as np
cars = pd.read_csv('./datasets/craiglist_cars.zip')


# <h1 id="exploration">Ingeniería de datos</h1>


## Evaluación de la cantidad, tipo y completitud de las variables disponibles. 

Se comenzará por reemplazar por la moda aquellas columnas que posean al menos un 10% de valores nulos.

In [6]:
# Diccionario que guarda los porcentajes de valores nulos en cada columna
nan_percent_values = dict(cars.isna().sum()/cars.shape[0] * 100)
nan_percent_values

{'condition': 36.860648682981164,
 'cylinders': 28.3063547367113,
 'drive': 13.280189256734676,
 'fuel': 0.8395045818355878,
 'manufacturer': 3.6461377726433803,
 'paint_color': 19.73457111165319,
 'price': 0.0,
 'size': 58.13362114594216,
 'title_status': 0.6629969024858356,
 'transmission': 0.9890873609455472,
 'type': 11.566660990191972,
 'year': 0.0}

In [9]:
# funcion para cambiar valores nulos por la moda
def nan_to_mode(df, nan_values_dict):
  values = list(nan_values_dict.values())
  keys = list(nan_values_dict.keys())
  
  for i in values:
      if i < float(10) and i != float(0):
        column = keys[values.index(i)]
        print(f"'{column}' has {round(i,2)}% of NaN values, I will replace it by the mode. \n")
        df[column] = df[column].fillna(df[column].mode()[0])
  return df

working_df = nan_to_mode(cars, nan_percent_values)

'manufacturer' has 3.65% of NaN values, I will replace it by the mode. 

'fuel' has 0.84% of NaN values, I will replace it by the mode. 

'title_status' has 0.66% of NaN values, I will replace it by the mode. 

'transmission' has 0.99% of NaN values, I will replace it by the mode. 



In [10]:
# verificacion del cambio
working_df.isna().sum()/working_df.shape[0] * 100

year             0.000000
manufacturer     0.000000
condition       36.860649
cylinders       28.306355
fuel             0.000000
title_status     0.000000
transmission     0.000000
drive           13.280189
size            58.133621
type            11.566661
paint_color     19.734571
price            0.000000
dtype: float64

## Transformación de los tipos de datos: One Hot Encoding

In [11]:
# tipos de datos existentes en el dataframe 
types = pd.DataFrame(data=working_df.dtypes, columns=['Variable types'])
types.groupby('Variable types').size()

Variable types
int64      2
object    10
dtype: int64

In [13]:
# seleccionar datos de tipo objeto
objects_list = list(types[types.values == 'object'].index)
objects_list

['manufacturer',
 'condition',
 'cylinders',
 'fuel',
 'title_status',
 'transmission',
 'drive',
 'size',
 'type',
 'paint_color']

A continuación se explora el contenido de cada columna con el fin de identificar que todos los datos a utilizar aporten valor al análisis. 

In [23]:
for column in objects_list:
  print(f"Column: {column}")
  print(f"Data: {list(working_df[column].value_counts().index)}")
  print()

Column: manufacturer
Data: ['ford', 'chevrolet', 'toyota', 'nissan', 'honda', 'jeep', 'ram', 'gmc', 'dodge', 'bmw', 'hyundai', 'subaru', 'mercedes-benz', 'volkswagen', 'chrysler', 'kia', 'cadillac', 'buick', 'lexus', 'mazda', 'audi', 'acura', 'infiniti', 'pontiac', 'lincoln', 'volvo', 'mitsubishi', 'mini', 'rover', 'mercury', 'saturn', 'jaguar', 'fiat', 'harley-davidson', 'datsun', 'alfa-romeo', 'land rover', 'aston-martin', 'porche', 'ferrari', 'hennessey', 'morgan']

Column: condition
Data: ['excellent', 'good', 'like new', 'fair', 'new', 'salvage']

Column: cylinders
Data: ['6 cylinders', '8 cylinders', '4 cylinders', '5 cylinders', '10 cylinders', 'other', '3 cylinders', '12 cylinders']

Column: fuel
Data: ['gas', 'diesel', 'other', 'hybrid', 'electric']

Column: title_status
Data: ['clean', 'rebuilt', 'salvage', 'lien', 'missing', 'parts only']

Column: transmission
Data: ['automatic', 'manual', 'other']

Column: drive
Data: ['4wd', 'fwd', 'rwd']

Column: size
Data: ['full-size', 

👉 Del código anterior se observa que las columnas `cylinders`, `fuel`, `transmission`, y `type`, tienen data etiquetada como `other`, lo cual **no aporta valor**. Por lo tanto, esta etiqueta será eliminada en el proceso final de One Hot Encoding. 

In [24]:
def OneHotEncoding_df(df, column):
  OHE_df = pd.get_dummies(column+'_'+df[column])
  return OHE_df

for category in objects_list:
  mask = OneHotEncoding_df(working_df, category)
  print(f'column {category} transformed!')
  working_df.drop(category, axis=1, inplace=True)
  working_df = pd.concat([working_df, (mask).astype(int)], axis=1)

print(f"Dataset final size: {working_df.shape}")

column manufacturer transformed!
column condition transformed!
column cylinders transformed!
column fuel transformed!
column title_status transformed!
column transmission transformed!
column drive transformed!
column size transformed!
column type transformed!
column paint_color transformed!
Dataset final size: (434542, 104)


In [25]:
working_df.drop(['cylinders_other', 'transmission_other', 'fuel_other', 'type_other'], axis=1, inplace=True)

In [26]:
working_df.shape

(434542, 100)

In [32]:
# por último verificamos el cambio total a variables categóricas
types = pd.DataFrame(data=working_df.dtypes, columns=['Variable types'])
types.groupby('Variable types').size()

Variable types
int64    100
dtype: int64

# Exportar DataFrame

In [29]:
working_df.to_parquet('./datasets/cars.parquet')